# BioInformatcis Project 5

## Import python packages

In [7]:
# %matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from time import time
import scipy.sparse as sp

from utils import *
from model import *

## Preprocess data

In [2]:
adj = load_data()
num_nodes = adj.shape[0]
num_edges = adj.sum()

# Featureless
features = sparse2tensor(sp.identity(num_nodes))
num_features = num_nodes
features_nonzero = num_nodes

print("Data load Success!!")

adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false, orig_matrix = split_data(adj)
adj_train_norm = normalize_graph(adj_train)

Data load Success!!
Data split Success!!
remain # of edges :  10609
remain # of edges :  256
remain # of edges :  4
val_edge_false generated!!
remain # of edges :  10609
remain # of edges :  264
remain # of edges :  4
test_edge_false generated!!


## Create & Fit model

In [3]:
# Create model
model = GCN(num_features, features_nonzero, num_h1=32, num_h2=16, dropout=0.1)

In [4]:
# Fit model
adj_label = adj_train + sp.eye(adj_train.shape[0])
adj_label = sparse2tensor(adj_label)

In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
pos_weight = (num_nodes**2 - num_edges) / num_edges
norm = num_nodes ** 2 / ((num_nodes**2 - num_edges) * 2)    
    
epochs=20
for epoch in range(epochs):
    tic = time()
    #print('Start of epoch %d' % (epoch,))
    inputs = [features, adj_train_norm]
    
    with tf.GradientTape() as tape:
        recon = model(inputs)        
        loss =  norm * tf.reduce_mean(
                        tf.nn.weighted_cross_entropy_with_logits(labels=tf.sparse.to_dense(adj_label),
                                                                logits=recon,
                                                                pos_weight=pos_weight))
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    
    #working on get_roc_score
    #roc_curr, ap_curr = get_roc_score(val_edges, val_edges_false)
    roc_curr = 1.
    ap_curr = 1.
    
    toc = int(time() - tic)
    print("Epoch:", '%04d' % (epoch + 1), 
          "train_loss=", "{:.5f}".format(loss),
          "val_roc=", "{:.5f}".format(roc_curr),
          "val_ap=", "{:.5f}".format(ap_curr),
          "time=", "{:.5f}".format(toc))

Epoch: 0001 train_loss= 0.61319 val_roc= 1.00000 val_ap= 1.00000 time= 1.00000
Epoch: 0002 train_loss= 0.61295 val_roc= 1.00000 val_ap= 1.00000 time= 1.00000
Epoch: 0003 train_loss= 0.61260 val_roc= 1.00000 val_ap= 1.00000 time= 1.00000
Epoch: 0004 train_loss= 0.61119 val_roc= 1.00000 val_ap= 1.00000 time= 1.00000
Epoch: 0005 train_loss= 0.61089 val_roc= 1.00000 val_ap= 1.00000 time= 1.00000
Epoch: 0006 train_loss= 0.61110 val_roc= 1.00000 val_ap= 1.00000 time= 1.00000
Epoch: 0007 train_loss= 0.61023 val_roc= 1.00000 val_ap= 1.00000 time= 1.00000
Epoch: 0008 train_loss= 0.60814 val_roc= 1.00000 val_ap= 1.00000 time= 1.00000
Epoch: 0009 train_loss= 0.60822 val_roc= 1.00000 val_ap= 1.00000 time= 1.00000
Epoch: 0010 train_loss= 0.60799 val_roc= 1.00000 val_ap= 1.00000 time= 1.00000
Epoch: 0011 train_loss= 0.60876 val_roc= 1.00000 val_ap= 1.00000 time= 1.00000
Epoch: 0012 train_loss= 0.60602 val_roc= 1.00000 val_ap= 1.00000 time= 1.00000
Epoch: 0013 train_loss= 0.60582 val_roc= 1.00000 val